In [8]:
from transformers import AutoTokenizer
from datasets import DatasetDict,Dataset,concatenate_datasets
import random

In [9]:
inputid_dataset= DatasetDict.load_from_disk("./../_data/datasets/inputid/bert-mult-uncased/")

In [10]:
inputid_test = inputid_dataset['test']

In [11]:
inputid_test

Dataset({
    features: ['sentences', 'tokens', 'labels', 'input_ids'],
    num_rows: 42419
})

In [12]:

# Define the error injection function
def inject_label_errors(dataset, error_rates):
    """
    Inject errors into the labels of the dataset.

    :param dataset: The input dataset.
    :param error_rates: A dictionary with error rates for each class.
                        Example: {0: 0.1, 1: 0.05, 2: 0.2}
    :return: A new dataset with injected label errors.
    """
    def apply_error(label):
        if random.random() < error_rates[label]:
            new_label = random.choice([l for l in range(len(error_rates)) if l != label])
            return new_label
        return label

    def inject_errors_in_labels(examples):
        examples["labels"] = [apply_error(label) for label in examples["labels"]]
        return examples

    # Apply error injection
    new_dataset = dataset.map(inject_errors_in_labels, batched=False)
    return new_dataset

error_rates = {0: 0.3, 1: 0.1, 2: 0.2}

# Inject errors into the dataset
injected_test= inject_label_errors(inputid_test, error_rates)

In [5]:
labeled_dataset = DatasetDict.load_from_disk("./../_data/datasets/labeled/all_sliced")
pretrained_model = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

/home/matin/Documents/python_environments/perspacor/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def tokenize_function(sent_tok):
    # Tokenize the input text
    tokens = sent_tok["tokens"]
    labels =  sent_tok["labels"]
    labels.insert(0, 0)  # Adding CLS token label at the beginning of each sequence
    labels.append(0)
    try:
        tokenized_sentence = [101] + [tokenizer.encode(tok)[1] for tok in tokens] + [102]
        sent_tok["input_ids"] = tokenized_sentence
        sent_tok["labels"] = labels
    except:
        print(len(tokens))
    return sent_tok
inputid_dataset = labeled_dataset.map(tokenize_function, load_from_cache_file=False) 
inputid_dataset

Map:   0%|          | 0/339344 [00:00<?, ? examples/s]

Map:   0%|          | 0/42418 [00:00<?, ? examples/s]

Map:   0%|          | 0/42419 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentences', 'tokens', 'labels', 'input_ids'],
        num_rows: 339344
    })
    validation: Dataset({
        features: ['sentences', 'tokens', 'labels', 'input_ids'],
        num_rows: 42418
    })
    test: Dataset({
        features: ['sentences', 'tokens', 'labels', 'input_ids'],
        num_rows: 42419
    })
})

In [7]:
inputid_dataset.save_to_disk("./../_data/datasets/inputid/bert-mult-uncased")

Saving the dataset (0/2 shards):   0%|          | 0/339344 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/42418 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/42419 [00:00<?, ? examples/s]

In [8]:
# Create an empty dictionary with specified columns
empty_data = { 
    "input_ids": [[1]],  # Replace with your actual input IDs
    "labels": [[0]],    # Replace with your actual labels
    "attention_mask": [[1]],  # Replace with your actual attention mask
}
bached_dataset = DatasetDict({
    "train": Dataset.from_dict(empty_data),
    "validation": Dataset.from_dict(empty_data),
    "test": Dataset.from_dict(empty_data)
})


In [9]:
from _utils.dataset_util import flatten_2d_list
def chunck_pad_map(sents_batch,dataset_name):
    
    f_tokens = flatten_2d_list(sents_batch["tokens"])
    f_labels = flatten_2d_list(sents_batch["labels"])
    f_input_ids = flatten_2d_list(sents_batch["input_ids"])
    # print(f_tokens)
    # print(len(f_input_ids))
    # print(len(f_labels))
    # # print(sents_batch["tokens"])
    # # print(len(sents_batch["tokens"]))
    # raise EOFError
    
    input_ids_list = []
    attention_mask_list = []
    labels_list = []
    chunk_size =512
    tok_pad=0 
    label_pad=-100
    attention_pad=0
    # Create chunks
    # print(len(f_input_ids)// chunk_size)
    for i in range(0, len(f_input_ids), chunk_size):  # We subtract 2 to account for special tokens
        chunked_tokens = f_input_ids[i:i + chunk_size]
        chunk_label_ids = f_labels[i:i + chunk_size]
        chunk_attention_mask = [1] * len(chunked_tokens)
        
        # print(len(chunked_tokens))
        
        while True and len(chunked_tokens) < chunk_size:
            chunked_tokens.append(tok_pad)
            chunk_attention_mask.append(attention_pad)
            chunk_label_ids.append(label_pad)
        input_ids_list.append(chunked_tokens)
        attention_mask_list.append(chunk_attention_mask)
        labels_list.append(chunk_label_ids)

    
    temp_ds = Dataset.from_dict({"input_ids": input_ids_list, "labels": labels_list, "attention_mask": attention_mask_list})
    bached_dataset[dataset_name] = bached_dataset[dataset_name].cast(temp_ds.features)
    bached_dataset[dataset_name]=concatenate_datasets([bached_dataset[dataset_name], temp_ds])
    return sents_batch


In [10]:

inputid_dataset["train"] = inputid_dataset["train"].remove_columns('sentences')
inputid_dataset["validation"] = inputid_dataset["validation"].remove_columns('sentences')
inputid_dataset["test"] = inputid_dataset["test"].remove_columns('sentences')

inputid_dataset["train"].map(chunck_pad_map,batched=True, load_from_cache_file=False,fn_kwargs={"dataset_name": "train"})
inputid_dataset["validation"].map(chunck_pad_map,batched=True, load_from_cache_file=False,fn_kwargs={"dataset_name": "validation"})
inputid_dataset["test"].map(chunck_pad_map,batched=True, load_from_cache_file=False,fn_kwargs={"dataset_name": "test"})

Map:   0%|          | 0/339344 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/220 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/451 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/667 [00:00<?, ? examples/s]

...

Casting the dataset:   0%|          | 0/8780 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9007 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9228 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9453 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'labels', 'input_ids'],
    num_rows: 42419
})

In [11]:
print(inputid_dataset)
print(bached_dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids'],
        num_rows: 339344
    })
    validation: Dataset({
        features: ['tokens', 'labels', 'input_ids'],
        num_rows: 42418
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids'],
        num_rows: 42419
    })
})
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 76796
    })
    validation: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 9565
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 9546
    })
})


In [12]:
bached_dataset['train'] = bached_dataset['train'].select(range(1,len(bached_dataset['train'])) )
bached_dataset['validation'] = bached_dataset['validation'].select(range(1,len(bached_dataset['validation'])) )
bached_dataset['test'] = bached_dataset['test'].select(range(1,len(bached_dataset['test'])) )
bached_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 76795
    })
    validation: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 9564
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 9545
    })
})

In [13]:
# inputid_dataset.save_to_disk("./../_data/datasets/batched/bert-mult-uncased")

In [14]:
bached_dataset.save_to_disk("./../_data/datasets/batched/bert-base-multilingual-uncased")

Saving the dataset (0/2 shards):   0%|          | 0/76795 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9564 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9545 [00:00<?, ? examples/s]